In [1]:
import cv2

In [3]:
# Load image
img = cv2.imread("C:/Users/migue/OneDrive/Ambiente de Trabalho/SEMESTER 4 - AI/Group Project/IMAGES TO TEST SHARPNESS/IMAGE3.jpg")

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Increase contrast using Histogram Equalization
equalized = cv2.equalizeHist(gray)

# Apply a sharpening kernel
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
sharp = cv2.filter2D(equalized, -1, kernel)

# Save the result
cv2.imwrite("C:/Users/migue/OneDrive/Ambiente de Trabalho/SEMESTER 4 - AI/Group Project/IMAGES TO TEST SHARPNESS/IMAGE3_ENHANCED.jpg", sharp)

True

# ✅Good Model to Upscale

Model name is Upscayl

In [11]:
import cv2

img = cv2.imread("C:/Users/migue/OneDrive/Ambiente de Trabalho/SEMESTER 4 - AI/Group Project/IMAGES TO TEST SHARPNESS/IMAGE3_upscayl_5x_upscayl-lite-4x.png")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
equalized = cv2.equalizeHist(gray)
cv2.imwrite("C:/Users/migue/OneDrive/Ambiente de Trabalho/SEMESTER 4 - AI/Group Project/IMAGES TO TEST SHARPNESS/IMAGE3_upscayl_upscaled.png", equalized)



True

## Testing

In [12]:
import cv2
import os

# Set your folder with original barcode images
input_folder = "C:/Users/migue/OneDrive/Ambiente de Trabalho/SEMESTER 4 - AI/Group Project/UPSCAYL/Barcodes"        # Change this to your folder name
output_folder = "C:/Users/migue/OneDrive/Ambiente de Trabalho/SEMESTER 4 - AI/Group Project/UPSCAYL/Preprocessed"   # Folder where processed images will be saved

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Supported image types
valid_extensions = [".jpg", ".jpeg", ".png", ".bmp"]

# Loop through all files
for filename in os.listdir(input_folder):
    if any(filename.lower().endswith(ext) for ext in valid_extensions):
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path)
        
        if img is None:
            print(f"❌ Could not read {filename}")
            continue
        
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        equalized = cv2.equalizeHist(gray)
        
        # Save processed image
        output_path = os.path.join(output_folder, os.path.splitext(filename)[0] + "_upscaled.jpg")
        cv2.imwrite(output_path, equalized)
        print(f"✅ Processed: {filename} → {output_path}")


✅ Processed: IMAGE1.jpg → C:/Users/migue/OneDrive/Ambiente de Trabalho/SEMESTER 4 - AI/Group Project/UPSCAYL/Preprocessed\IMAGE1_upscaled.jpg
✅ Processed: IMAGE2.jpg → C:/Users/migue/OneDrive/Ambiente de Trabalho/SEMESTER 4 - AI/Group Project/UPSCAYL/Preprocessed\IMAGE2_upscaled.jpg
✅ Processed: IMAGE3.jpg → C:/Users/migue/OneDrive/Ambiente de Trabalho/SEMESTER 4 - AI/Group Project/UPSCAYL/Preprocessed\IMAGE3_upscaled.jpg
✅ Processed: IMAGE4.jpg → C:/Users/migue/OneDrive/Ambiente de Trabalho/SEMESTER 4 - AI/Group Project/UPSCAYL/Preprocessed\IMAGE4_upscaled.jpg


# Barcode detection

In [ ]:
import cv2
import requests
import os
from PIL import Image
import imagehash

# --- Roboflow API Setup ---
API_KEY = "riK5t7eDoQp67Y2kXQGW"
MODEL_ID = "barcode-detection-brbvg"
VERSION = "2"
API_URL = f"https://detect.roboflow.com/{MODEL_ID}/{VERSION}?api_key={API_KEY}"

# --- Video Input ---
video_path = r"C:\Users\Dell\Downloads\0506-1.mp4"
cap = cv2.VideoCapture(video_path)

# --- Check video opened ---
if not cap.isOpened():
    print("[ERROR] Cannot open video.")
    exit()

frame_count = 0
frame_skip = 10  # Process every 10th frame for performance

# Create folder for cropped images
crop_dir = "cropped"
os.makedirs(crop_dir, exist_ok=True)

# Set to keep track of saved image hashes
saved_hashes = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("[ERROR] Could not read next frame. Exiting...")
        break

    # Skip frames
    if frame_count % frame_skip != 0:
        frame_count += 1
        continue

    # Resize for API
    resized = cv2.resize(frame, (640, 480))

    # Save resized frame temporarily
    temp_image_path = "temp.jpg"
    cv2.imwrite(temp_image_path, resized)

    # Send to Roboflow API
    try:
        with open(temp_image_path, "rb") as img_file:
            response = requests.post(API_URL, files={"file": img_file})
        data = response.json()
        predictions = data.get("predictions", [])
        print(f"[OK] Frame {frame_count}: {len(predictions)} barcode(s) found.")
    except Exception as e:
        print("[ERROR] API call failed:", e)
        break

    # Delete temp file
    if os.path.exists(temp_image_path):
        os.remove(temp_image_path)

    # Process predictions
    for i, pred in enumerate(predictions):
        x = int(pred["x"])
        y = int(pred["y"])
        w = int(pred["width"])
        h = int(pred["height"])
        label = pred["class"]

        # Coordinates
        x1 = max(0, x - w // 2)
        y1 = max(0, y - h // 2)
        x2 = min(resized.shape[1], x + w // 2)
        y2 = min(resized.shape[0], y + h // 2)

        # Crop region
        cropped = resized[y1:y2, x1:x2]
        if cropped.size == 0:
            continue  # Skip invalid crops

        # Convert to PIL and compute hash
        pil_img = Image.fromarray(cropped)
        crop_hash = imagehash.average_hash(pil_img)

        # Fuzzy deduplication: allow some small differences
        is_unique = all(abs(crop_hash - existing_hash) > 5 for existing_hash in saved_hashes)

        if is_unique:
            saved_hashes.append(crop_hash)
            cropped_filename = os.path.join(crop_dir, f"barcode_{frame_count}_{i}.jpg")
            pil_img.save(cropped_filename)
            print(f"[CROP] Saved: {cropped_filename}")
        else:
            print("[SKIP] Similar barcode already saved.")

        # Draw bounding box
        cv2.rectangle(resized, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(resized, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Show result
    cv2.imshow("Barcode Detection", resized)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("[STOP] Stopped by user.")
        break

    frame_count += 1

cap.release()
cv2.destroyAllWindows()
print("[DONE] Finished video.")
